In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Info-stats-ai/End-to-End-ML-Wine-Quality-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Info-stats-ai"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3f7cd5fa28a9b157d3b03d0a7840051ae1e88b55"

In [2]:

%pwd

'/Users/omkarthakur/Desktop/Ds_project/research'

In [3]:
os.chdir("../")
%pwd

'/Users/omkarthakur/Desktop/Ds_project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="file:///tmp/mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # FIXED: Skip model logging entirely for Dagshub
            if tracking_url_type_store == "file":
                # Only log model if using local file storage
                mlflow.sklearn.log_model(model, "model")
            else:
                # For Dagshub, just log a note that model is saved locally
                mlflow.log_text(f"Model saved locally at: {self.config.model_path}", "model_info.txt")
                print(f"Model saved locally at: {self.config.model_path}")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-14 00:42:40,489: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-09-14 00:42:40,491: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-14 00:42:40,493: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-14 00:42:40,494: INFO: common: created directory at: artifacts]
[2025-09-14 00:42:40,494: INFO: common: created directory at: artifacts/model_evaluation]
[2025-09-14 00:42:40,681: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
Model saved locally at: artifacts/model_trainer/model.joblib
🏃 View run righteous-chimp-922 at: https://dagshub.com/Info-stats-ai/End-to-End-ML-Wine-Quality-Prediction.mlflow/#/experiments/0/runs/38ab2cdc29a14d499142b087632a06fc
🧪 View experiment at: https://dagshub.com/Info-stats-ai/End-to-End-ML-Wine-Quality-Prediction.mlflow/#/experiments/0
